In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

/Users/andrew/opt/anaconda3/envs/rp2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset('hotpotqa/hotpot_qa', 'fullwiki', trust_remote_code=True)
train_dataset = ds['train']
def preprocess(example):
    context="Context: "
    question = example['question']
    for title, passage in zip(example['context']['title'], example['context']['sentences']):
        sentences = " ".join(passage)
        context+=f"{title} : {sentences}"
        context+="\n"
    context+=f"question: {question}\nevidences: "
    example = {
        'input': context,
        'answer': str(example['supporting_facts']['title']),
    }
    return example
# ['question', 'answer', 'level','supporting_facts', 'context'])

train_dataset_mapped = train_dataset.map(preprocess)
train_dataset_mapped = train_dataset_mapped.select_columns(['input', 'answer'])

Map: 100%|██████████| 90447/90447 [00:15<00:00, 5758.11 examples/s]


In [3]:
# train_dataset_mapped[0:4]

In [3]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/andrew/opt/anaconda3/envs/rp2/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
import torch
def tokenize(batch):
    batch_encoding = tokenizer(
        batch['input'],
        padding="longest",
        return_tensors='pt'
    )
    target_encoding = tokenizer(
        batch['answer'],
        padding="longest",
        return_tensors = 'pt'
    )
    labels = target_encoding.input_ids
    labels[labels == tokenizer.pad_token_id] = -100
    return {
        'input_ids': torch.tensor(batch_encoding['input_ids']),
        'attention_mask': torch.tensor(batch_encoding['attention_mask']),
        'labels': torch.tensor(labels)
    }

In [15]:
encoded_dataset=train_dataset_mapped.map(tokenize, batch_size=4, batched=True)

Map:   0%|          | 0/90447 [00:00<?, ? examples/s]/var/folders/0r/6ncbx7693d96pr3xr0xpgt8w0000gn/T/ipykernel_70161/3795642879.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_encoding['input_ids']),
/var/folders/0r/6ncbx7693d96pr3xr0xpgt8w0000gn/T/ipykernel_70161/3795642879.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(batch_encoding['attention_mask']),
/var/folders/0r/6ncbx7693d96pr3xr0xpgt8w0000gn/T/ipykernel_70161/3795642879.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor

In [ ]:
encoded_dataset = encoded_dataset.remove_columns(['input', 'answer'])